In [1]:
import pandas as pd

In [ ]:
tickets_df: pd.DataFrame = pd.DataFrame(
    columns=(
        "event_id",
        "event_date",
        "event_type",
        "event_multiday",
        "event_class",
        "venue_type",
        "venue_cap",
        "venue_lat",
        "venue_lon",
        "promoter_tier",
        "days_until_event",
        "ticket_sell_date",
        "ticket_sell_price",
    )
)

df = tickets_df.groupby(["event_id", "ticket_sale_date"]).agg({
    'ticket_sell_date': 'size',
    'event_typeA': 'first',
    'event_multiday': 'first',
    'event_class': 'first',
    'venue_type': 'first',
    'venue_cap': 'first',
    'venue_lat': 'first',
    'venue_lon': 'first',
    'promoter_tier': 'first',
    'days_until_event': 'first'
}).reset_index()

df.rename(
    columns={'ticket_sell_date': 'date', 'ticket_sell_date': 'sales'},
    inplace=True
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import keras as kr

In [ ]:
X = df.drop(columns=['sales', 'event_id', 'date'])
Y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

categorical_features = [
    'event_type', 'event_multiday', 'event_class', 'venue_type'
]
numeric_features = [
    'venue_cap', 'venue_lat', 'venue_lon', 'promoter_tier', 'days_until_event'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
model: kr.models.Sequential = kr.models.Sequential([
    kr.layers.Input(shape=(X_train.shape[1],)),
    kr.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    kr.layers.Dense(64, activation='relu'),
    kr.layers.Dense(32, activation='relu'),
    kr.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50, validation_split=0.2, batch_size=32,
    verbose=0
)

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
predictions = model.predict(X_test)

for i in range(10):
    print(f'Predicted: {predictions[i][0]}, Actual: {y_test.iloc[i]}')